In [1]:
import re
import csv
import glob
import numpy
from scipy import stats

In [63]:
#path_a="data/nlp/evaluation/outputs2/bart-base-kp20k-15epochs.pre.csv"
#path_a="data/astro/evaluation/outputs2/bart-base-kp20k-15epochs.pre.csv"

model = "bart-large"
domain = "paleo"
split = "all"

path_a="data/{}/outputs/{}.{}.csv".format(domain, model, split)
#path_a="data/others/kp20k/bart-ft-15-kp20k.all.csv"
#path_a="data/others/kp20k/one2set.kp20k.abs.csv"
column_id = 3

In [64]:
def load_scores(input_csv_file, col_id=1):
    keys, scores = [], []
    with open(input_csv_file, 'r', newline='') as csvfile:
        reference_reader = csv.reader(csvfile, delimiter='\t')
        for row in reference_reader:
            keys.append(row[0])
            scores.append(float(row[col_id]))
    return (keys, scores)

In [65]:
scores_a = load_scores(path_a, col_id=column_id)
path_dir = "/".join(path_a.split("/")[:-1])

print(path_dir)
score_reference = numpy.average(scores_a[1])
print("reference avg: {0:.4f}".format(score_reference))

for input_file in glob.glob(path_dir+'/*.{}.csv'.format(split)):
    if input_file == path_a:
        continue
    scores_b = load_scores(input_file, col_id=column_id)
    assert(scores_a[0] == scores_b[0])
    
    if numpy.average(scores_b[1]) < score_reference:
        continue
        
    ttest = stats.ttest_rel(a=scores_a[1], b=scores_b[1])
    if ttest[1] < .05:
        print('avg: {0:.4f} (sign@.05: {1}, pvalue: {2:.8f}) - {3}'.format(numpy.average(scores_b[1]), ttest[1] < .05, ttest[1], input_file.split("/")[-1]))
    #print('avg: {0:.4f} (sign@.05: {1}, pvalue: {2:.8f}) - {3}'.format(numpy.average(scores_b[1]), ttest[1] < .05, ttest[1], input_file.split("/")[-1]))
    #print(input_file)



data/paleo/outputs
reference avg: 0.1910
avg: 0.2004 (sign@.05: True, pvalue: 0.02775842) - bart-large-paleo-self-1000-3epochs.lr1e-6.all.csv
avg: 0.2010 (sign@.05: True, pvalue: 0.01235018) - bart-large-paleo-500-3epochs.lr1e-6.all.csv
